In [1]:
# dependencies
import pandas as pd

In [2]:
df1 = pd.read_csv("data/JC-202101-citibike-tripdata.csv")
df2 = pd.read_csv("data/JC-202002-citibike-tripdata.csv")
df3 = pd.read_csv("data/JC-202003-citibike-tripdata.csv")
df4 = pd.read_csv("data/JC-202004-citibike-tripdata.csv")
df5 = pd.read_csv("data/JC-202005-citibike-tripdata.csv")
df6 = pd.read_csv("data/JC-202006-citibike-tripdata.csv")
df7 = pd.read_csv("data/JC-202007-citibike-tripdata.csv")
df8 = pd.read_csv("data/JC-202008-citibike-tripdata.csv")
df9 = pd.read_csv("data/JC-202009-citibike-tripdata.csv")
df10 = pd.read_csv("data/JC-202010-citibike-tripdata.csv")
df11 = pd.read_csv("data/JC-202011-citibike-tripdata.csv")
df12 = pd.read_csv("data/JC-202012-citibike-tripdata.csv")

In [3]:
frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12]
# concatenate the list of dataframes to a single dataframe
df = pd.concat(frames, sort=False)
df = df.drop_duplicates()

# Drop rows where the trip duration is less than 90 seconds and where the start and end stations are the same
# Filter out records where users rented a bike but found that it wasn't working properly
df = df.loc[((df["tripduration"]>90) & (df["start station id"] != df["end station id"])),:]

# Convert trip duration column from seconds to minutes
df["tripduration"] = [trip/60 for trip in df["tripduration"]]

#select columns
citi_df = df[["tripduration", "starttime", "stoptime", "start station id", "start station name", "start station latitude", "start station longitude", "end station id", "end station name", "end station latitude", "end station longitude", "bikeid", "usertype", "birth year", "gender"]] 
citi_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,4.433333,2021-01-01 00:03:35.5100,2021-01-01 00:08:01.7770,3273,Manila & 1st,40.721651,-74.042884,3209,Brunswick St,40.724176,-74.050656,42494,Subscriber,1988,1
1,25.716667,2021-01-01 00:23:32.9250,2021-01-01 00:49:16.0830,3681,Grand St,40.715178,-74.037683,3213,Van Vorst Park,40.718489,-74.047727,45343,Customer,1996,2
2,24.350000,2021-01-01 00:23:50.7940,2021-01-01 00:48:12.5660,3681,Grand St,40.715178,-74.037683,3213,Van Vorst Park,40.718489,-74.047727,31794,Customer,1995,1
3,13.216667,2021-01-01 00:31:09.0770,2021-01-01 00:44:22.9430,3185,City Hall,40.717732,-74.043845,3199,Newport Pkwy,40.728745,-74.032108,42316,Customer,1969,0
4,9.933333,2021-01-01 00:35:52.1900,2021-01-01 00:45:48.7740,3639,Harborside,40.719252,-74.034234,3209,Brunswick St,40.724176,-74.050656,32575,Customer,1969,0


In [4]:
from math import sin, cos, sqrt, atan2, radians

# Convert degrees to radians
lat1 = [radians(lat) for lat in df["start station latitude"]]
lon1 = [radians(lon) for lon in df["start station longitude"]]
lat2 = [radians(lat) for lat in df["end station latitude"]]
lon2 = [radians(lon) for lon in df["end station longitude"]]

# Convert lists into series
lat1 = pd.Series(lat1)
lon1 = pd.Series(lon1)
lat2 = pd.Series(lat2)
lon2 = pd.Series(lon2)

# Calculate difference between each set of latitude and longitude
dlat = lat2 - lat1
dlon = lon2 - lon1

In [5]:
# Approximate radius of Earth in km
R = 6373.0

# Empty list to store trip distances
distance = []

for i in range(0, len(lat1)):
    
    a = sin(dlat[i] / 2)**2 + cos(lat1[i]) * cos(lat2[i]) * sin(dlon[i] / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Get distance and convert km to miles
    miles = (R * c) * .6214
    
    # Append miles travel to 'distance' list
    distance.append(miles)

In [6]:
# Add trip distance as new column to data frame
citi_df["tripdistance (mi)"] = distance

In [7]:
citi_df["gender"].replace({0: "unknown", 1: "Male", 2: "Female"}, inplace=True)

In [8]:
# Add ride id column to uniquely identify each ride
citi_df["rideid"] = citi_df["start station id"].map(str) + "_" + citi_df["end station id"].map(str)

In [9]:
# Calculate age of riders
ages = []

for i in citi_df["birth year"]:
    age = 2020 - i
    ages.append(age)

In [10]:
# Append new Age column
citi_df["Age"] = ages

In [11]:
# Delete all rows where user age is over 90
citi_df = citi_df.loc[citi_df["Age"] < 90]

In [12]:
# Divide users in age bins
bins = [0, 17, 25, 35, 50, 65, 90]
labels = ["0-17", "18-25", "26-35", "36-50", "51-65", "65-90"]

citi_df["Age Group"] = pd.cut(citi_df["Age"], bins, labels=labels)
citi_df.head(5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,tripdistance (mi),rideid,Age,Age Group
0,4.433333,2021-01-01 00:03:35.5100,2021-01-01 00:08:01.7770,3273,Manila & 1st,40.721651,-74.042884,3209,Brunswick St,40.724176,-74.050656,42494,Subscriber,1988,Male,0.442985,3273_3209,32,26-35
1,25.716667,2021-01-01 00:23:32.9250,2021-01-01 00:49:16.0830,3681,Grand St,40.715178,-74.037683,3213,Van Vorst Park,40.718489,-74.047727,45343,Customer,1996,Female,0.573768,3681_3213,24,18-25
2,24.350000,2021-01-01 00:23:50.7940,2021-01-01 00:48:12.5660,3681,Grand St,40.715178,-74.037683,3213,Van Vorst Park,40.718489,-74.047727,31794,Customer,1995,Male,0.573768,3681_3213,25,18-25
3,13.216667,2021-01-01 00:31:09.0770,2021-01-01 00:44:22.9430,3185,City Hall,40.717732,-74.043845,3199,Newport Pkwy,40.728745,-74.032108,42316,Customer,1969,unknown,0.978435,3185_3199,51,51-65
4,9.933333,2021-01-01 00:35:52.1900,2021-01-01 00:45:48.7740,3639,Harborside,40.719252,-74.034234,3209,Brunswick St,40.724176,-74.050656,32575,Customer,1969,unknown,0.925164,3639_3209,51,51-65


In [13]:
citi_df.to_csv("data/2020-citibike.csv", index=False)